In [23]:
import theano
from theano import tensor

import numpy
import cPickle as pkl

import matplotlib.pylab as plot
from matplotlib.cm import gray

In [24]:
import nmt

In [25]:
model = "/home/sebastien/Documents/Git/doc-nmt/doc2/model.doc2.npz"

In [26]:
# load model model_options
with open('%s.pkl' % model, 'rb') as f:
    options = pkl.load(f)

# load source dictionary and invert
with open(options['dictionaries'][0], 'rb') as f:
    word_dict = pkl.load(f)
word_idict = dict()
for kk, vv in word_dict.iteritems():
    word_idict[vv] = kk
word_idict[0] = '<eos>'
word_idict[1] = 'UNK'

# load target dictionary and invert
with open(options['dictionaries'][1], 'rb') as f:
    word_dict_trg = pkl.load(f)
word_idict_trg = dict()
for kk, vv in word_dict_trg.iteritems():
    word_idict_trg[vv] = kk
word_idict_trg[0] = '<eos>'
word_idict_trg[1] = 'UNK'

In [27]:
theano.config.floatX='float32'

print 'Building model'
params = nmt.init_params(options)
# reload parameters
params = nmt.load_params(model, params)
tparams = nmt.init_tparams(params)

trng, use_noise, \
    x, x_mask, y, y_mask, xc, xc_mask, \
    opt_ret, \
    cost, xc_mask_2, xc_mask_3 = \
    nmt.build_model(tparams, options)
inps = [x, x_mask, y, y_mask, xc, xc_mask, xc_mask_2, xc_mask_3]

Building model


In [28]:
print 'Building sampler'
f_init, f_next = nmt.build_sampler(tparams, options, trng, use_noise)

Building sampler
Building f_init... Done
Building f_next.. Done


In [29]:
print 'Building f_ctx_att...',
f_ctx_att = theano.function(inps, opt_ret['ctx_alphas'], profile=False)
print 'Done'

Building f_ctx_att... Done


In [30]:
#words = line.strip().split()
#context_words = gline.strip().split(' ||| ') # with |||

words = 'And just as the bi@@ osphere has been sever@@ ely ero@@ ded , so too is the ethn@@ osphere -- and , if anything , at a far greater rate .'
words = words.strip().split()

context_words = 'Now , together the my@@ ri@@ ad cultures of the world make up a web of spiritual life and cultural life that envel@@ ops the planet , and is as important to the well-being of the planet as indeed is the biological web of life that you know as a bi@@ osphere . ||| And you might think of this cultural web of life as being an ethn@@ osphere , and you might define the ethn@@ osphere as being the sum total of all thoughts and dreams , my@@ ths , ideas , inspi@@ rations , intui@@ tions brought into being by the human imagination since the dawn of consciousness . ||| The ethn@@ osphere is humanity &apos;s great legacy . ||| It &apos;s the symbol of all that we are and all that we can be as an astoni@@ sh@@ ingly in@@ qui@@ si@@ tive species . ||| No biologists , for example , would d@@ are suggest that 50 percent of all species or more have been or are on the br@@ ink of extinction because it simply is not true , and yet that -- the most ap@@ oc@@ al@@ yp@@ tic scenario in the realm of biological diversity -- scar@@ ce@@ ly approaches what we know to be the most optimistic scenario in the realm of cultural diversity . ||| And the great indic@@ ator of that , of course , is language loss . ||| When each of you in this room were born , there were 6,000 languages spoken on the planet . ||| Now , a language is not just a body of voc@@ ab@@ ul@@ ary or a set of gra@@ m@@ ma@@ tical rules .'
context_words = context_words.strip().split(' ||| ')


x = map(lambda w: word_dict[w] if w in word_dict else 1, words)
x = map(lambda ii: ii if ii < options['n_words_src'] else 1, x)
x += [0]
x = numpy.array(x).reshape([len(x), 1])

num_sent = len(context_words)
maxlen = max(len(sent) for sent in context_words) + 1
xc = numpy.zeros((num_sent, 1, maxlen), dtype=numpy.int64)
xc_mask = numpy.zeros((num_sent, 1, maxlen), dtype=numpy.float32)
xc_mask_2 = numpy.zeros((num_sent, 1), dtype=numpy.float32)
xc_mask_3 = numpy.zeros((num_sent, 1), dtype=numpy.float32)

for ii, sent in enumerate(context_words):
    sent = sent.strip().split()
    tmp = map(lambda w: word_dict[w] if w in word_dict else 1, sent)
    tmp = map(lambda ii: ii if ii < options['n_words_src'] else 1, tmp)
    tmp += [0]
    xc[ii,0,:len(tmp)] = tmp
    xc_mask[ii,0,:len(tmp)] = 1.0
    xc_mask_2[ii,0] = 1.0
    xc_mask_3[ii,0] = float(len(tmp))

In [31]:
sample, score = nmt.gen_sample(tparams, f_init, f_next,
                           x, xc, xc_mask, xc_mask_2, xc_mask_3,
                           options, trng=trng, k=1, maxlen=200,
                           stochastic=False, argmax=False)

In [32]:
y = numpy.array(sample[0]).reshape([len(sample[0]), 1])

In [33]:
ctx_att = f_ctx_att(x, numpy.float32(x*0.+1.), y, numpy.float32(y*0.+1.), xc, xc_mask, xc_mask_2, xc_mask_3)

In [34]:
trans = [word_idict_trg[t] for t in sample[0]]
print ' '.join(trans)

Et tout comme la bi@@ osphère a été sév@@ èrement éro@@ positive , aussi aussi , c&apos; est l&apos; ethni@@ osphère -- et , si quelque chose , à un taux bien plus grand . <eos>


In [35]:
ctx_att = ctx_att.reshape(ctx_att.shape[0], ctx_att.shape[2])

In [36]:
print ctx_att

[[ 0.12531212  0.12523271  0.12499771  0.12511538  0.12514547  0.12486754
   0.12542093  0.1239082 ]
 [ 0.12531106  0.12523156  0.12499743  0.12511668  0.12514172  0.12487195
   0.12541819  0.12391135]
 [ 0.12532325  0.12523501  0.12499639  0.12511928  0.12512918  0.12487633
   0.12541507  0.12390553]
 [ 0.12531275  0.12522399  0.12500586  0.12511858  0.12514704  0.12485819
   0.12541457  0.12391894]
 [ 0.12531124  0.12523231  0.12499608  0.12510929  0.12514047  0.12487021
   0.12542485  0.12391561]
 [ 0.12531738  0.12523398  0.12498699  0.12511024  0.125144    0.12486845
   0.12542981  0.12390918]
 [ 0.12531106  0.1252314   0.12499857  0.1251166   0.12514508  0.12486483
   0.12542365  0.12390873]
 [ 0.12530532  0.12522614  0.12500775  0.12511346  0.12514186  0.12486653
   0.1254209   0.12391802]
 [ 0.12530941  0.12523769  0.1249994   0.12510854  0.12515031  0.12485349
   0.12543282  0.12390837]
 [ 0.12531456  0.12523213  0.1249951   0.12511158  0.12514111  0.12487276
   0.12542357  0.